In [ ]:
!pip install pytorch-metric-learning
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import argparse
import torch
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from pytorch_metric_learning import samplers
import logging
import time
import pdb
import os
import json
import random
from tqdm import tqdm

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/sapBERT-DUT-cambridge")

In [ ]:
from src.data_loader import (
    DictionaryDataset,
    QueryDataset,
    QueryDataset_pretraining,
    MetricLearningDataset,
    MetricLearningDataset_pairwise,
)
from src.model_wrapper import (
    Model_Wrapper
)
from src.metric_learning import (
    Sap_Metric_Learning,
)

LOGGER = logging.getLogger()


In [ ]:
epochs = 1

args = {
    'epoch' : epochs,
    'train_dir' : '/content/drive/MyDrive/sapBERT-DUT-cambridge/training_data/training_file_umls2020aa_en_uncased_no_dup_pairwise_pair_th50_filtered-categories.txt',
    'model_dir' : 'CLTL/MedRoBERTa.nl',
    'output_dir' : f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{epochs}_epoch/ft_0_epoch',
    'use_cuda' : True,
    'learning_rate' : 0.0001,
    'weight_decay' : 0.01,
    'max_length' : 25,
    'train_batch_size' : 512,
    'checkpoint_step' : 10000000,
    'pairwise' : True,
    'amp' : True,
    'random_seed' : 1993,
    'loss' : 'ms_loss',
    'use_miner' : True,
    'miner_margin' : 0.2,
    'type_of_triplets' : 'all',
    'agg_mode' : 'cls',
    'save_checkpoint_all' : True
}

In [ ]:
def init_logging():
    LOGGER.setLevel(logging.INFO)
    fmt = logging.Formatter('%(asctime)s: [ %(message)s ]',
                            '%m/%d/%Y %I:%M:%S %p')
    console = logging.StreamHandler()
    console.setFormatter(fmt)
    LOGGER.addHandler(console)


In [ ]:
def init_seed(seed=None):
    if seed is None:
        seed = int(round(time.time() * 1000)) % 10000

    LOGGER.info("Using seed={}, pid={}".format(seed, os.getpid()))
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


In [ ]:
def load_dictionary(dictionary_path):
    """
    load dictionary

    Parameters
    ----------
    dictionary_path : str
        a path of dictionary
    """
    dictionary = DictionaryDataset(
            dictionary_path = dictionary_path
    )

    return dictionary.data

In [ ]:
def load_queries(data_dir, filter_composite, filter_duplicate):
    """
    load query data

    Parameters
    ----------
    is_train : bool
        train or dev
    filter_composite : bool
        filter composite mentions
    filter_duplicate : bool
        filter duplicate queries
    """
    dataset = QueryDataset(
        data_dir=data_dir,
        filter_composite=filter_composite,
        filter_duplicate=filter_duplicate
    )

    return dataset.data


In [ ]:
def load_queries_pretraining(data_dir, filter_duplicate):
    """
    load query data

    Parameters
    ----------
    is_train : bool
        train or dev
    filter_composite : bool
        filter composite mentions
    filter_duplicate : bool
        filter duplicate queries
    """
    dataset = QueryDataset_pretraining(
        data_dir=data_dir,
        filter_duplicate=filter_duplicate
    )

    return dataset.data


In [ ]:
def train(args, data_loader, model, scaler=None, model_wrapper=None, step_global=0):
    LOGGER.info("train!")

    train_loss = 0
    train_steps = 0
    model.cuda()
    model.train()
    for i, data in tqdm(enumerate(data_loader), total=len(data_loader)):
        model.optimizer.zero_grad()

        batch_x1, batch_x2, batch_y = data
        batch_x_cuda1, batch_x_cuda2 = {},{}
        for k,v in batch_x1.items():
            batch_x_cuda1[k] = v.cuda()
        for k,v in batch_x2.items():
            batch_x_cuda2[k] = v.cuda()

        batch_y_cuda = batch_y.cuda()

        if args['amp']:
            with autocast():
                loss = model(batch_x_cuda1, batch_x_cuda2, batch_y_cuda)
        else:
            loss = model(batch_x_cuda1, batch_x_cuda2, batch_y_cuda)
        if args['amp']:
            scaler.scale(loss).backward()
            scaler.step(model.optimizer)
            scaler.update()
        else:
            loss.backward()
            model.optimizer.step()

        train_loss += loss.item()
        # wandb.log({"Loss": loss.item()})
        train_steps += 1
        step_global += 1
        #if (i+1) % 10 == 0:
        #LOGGER.info ("epoch: {} loss: {:.3f}".format(i+1,train_loss / (train_steps+1e-9)))
        #LOGGER.info ("epoch: {} loss: {:.3f}".format(i+1, loss.item()))

        # save model every K iterations
        if step_global % args['checkpoint_step'] == 0:
            checkpoint_dir = os.path.join(args['output_dir'], "checkpoint_iter_{}".format(str(step_global)))
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            model_wrapper.save_model(checkpoint_dir)
    train_loss /= (train_steps + 1e-9)
    return train_loss, step_global


In [ ]:
def main(args):
    init_logging()

    torch.manual_seed(args['random_seed'])

    # prepare for output
    if not os.path.exists(args['output_dir']):
        os.makedirs(args['output_dir'])


    # load BERT tokenizer, dense_encoder
    model_wrapper = Model_Wrapper()
    encoder, tokenizer = model_wrapper.load_bert(
        path=args['model_dir'],
        max_length=args['max_length'],
        use_cuda=args['use_cuda'],
        #lowercase=not args.cased
    )

    # load SAP model
    model = Sap_Metric_Learning(
            encoder = encoder,
            learning_rate=args['learning_rate'],
            weight_decay=args['weight_decay'],
            use_cuda=args['use_cuda'],
            pairwise=args['pairwise'],
            loss=args['loss'],
            use_miner=args['use_miner'],
            miner_margin=args['miner_margin'],
            type_of_triplets=args['type_of_triplets'],
            agg_mode=args['agg_mode'],
    )

    # if args.parallel:
    #     model.encoder = torch.nn.DataParallel(model.encoder)
    #     LOGGER.info("using nn.DataParallel")

    def collate_fn_batch_encoding(batch):
        query1, query2, query_id = zip(*batch)
        query_encodings1 = tokenizer.batch_encode_plus(
                list(query1),
                max_length=args['max_length'],
                padding="max_length",
                truncation=True,
                add_special_tokens=True,
                return_tensors="pt")
        query_encodings2 = tokenizer.batch_encode_plus(
                list(query2),
                max_length=args['max_length'],
                padding="max_length",
                truncation=True,
                add_special_tokens=True,
                return_tensors="pt")
        #query_encodings_cuda = {}
        #for k,v in query_encodings.items():
        #    query_encodings_cuda[k] = v.cuda()
        query_ids = torch.tensor(list(query_id))
        return  query_encodings1, query_encodings2, query_ids

    if args['pairwise']:
        train_set = MetricLearningDataset_pairwise(
                path=args['train_dir'],
                tokenizer = tokenizer
        )
        train_loader = torch.utils.data.DataLoader(
            train_set,
            batch_size=args['train_batch_size'],
            shuffle=True,
            num_workers=16,
            collate_fn=collate_fn_batch_encoding
        )
    else:
        train_set = MetricLearningDataset(
            path=args['train_dir'],
            tokenizer = tokenizer
        )
        # using a sampler
        train_loader = torch.utils.data.DataLoader(
            train_set,
            batch_size=args['train_batch_size'],
            #shuffle=True,
            sampler=samplers.MPerClassSampler(train_set.query_ids,\
                2, length_before_new_iter=100000),
            num_workers=16,
            )
    # mixed precision training
    if args['amp']:
        scaler = GradScaler()
    else:
        scaler = None

    start = time.time()
    step_global = 0
    for epoch in range(1,args['epoch']+1):
        # embed dense representations for query and dictionary for train
        # Important! This is iterative process because dense represenation changes as model is trained.
        LOGGER.info("Epoch {}/{}".format(epoch,args['epoch']))

        # train
        train_loss, step_global = train(args, data_loader=train_loader, model=model, scaler=scaler, model_wrapper=model_wrapper, step_global=step_global)
        LOGGER.info('loss/train_per_epoch={}/{}'.format(train_loss,epoch))

        # save model every epoch
        if args['save_checkpoint_all']:
            checkpoint_dir = os.path.join(args['output_dir'], "checkpoint_{}".format(epoch))
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            model_wrapper.save_model(checkpoint_dir)

        # save model last epoch
        if epoch == args['epoch']:
            model_wrapper.save_model(args['output_dir'])

    end = time.time()
    training_time = end-start
    training_hour = int(training_time/60/60)
    training_minute = int(training_time/60 % 60)
    training_second = int(training_time % 60)
    LOGGER.info("Training Time!{} hours {} minutes {} seconds".format(training_hour, training_minute, training_second))


In [ ]:
main(args)

{'train_dir': '/content/drive/MyDrive/sapBERT-DUT-cambridge/training_data/training_file_umls2020aa_en_uncased_no_dup_pairwise_pair_th50_filtered-categories.txt', 'model_dir': 'CLTL/MedRoBERTa.nl', 'output_dir': '/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/1_epoch/ft_0_epoch', 'use_cuda': True, 'learning_rate': 0.0001, 'weight_decay': 0.01, 'max_length': 25, 'train_batch_size': 512, 'epoch': 1, 'checkpoint_step': 10000000, 'pairwise': True, 'amp': True, 'random_seed': 1993, 'loss': 'ms_loss', 'use_miner': True, 'miner_margin': 0.2, 'type_of_triplets': 'all', 'agg_mode': 'cls', 'save_checkpoint_all': True}


Some weights of RobertaModel were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:src.metric_learning:Sap_Metric_Learning! learning_rate=0.0001 weight_decay=0.01 use_cuda=True loss=ms_loss use_miner=True miner_margin=0.2 type_of_triplets=all agg_mode=cls
10/31/2023 11:54:41 AM: [ Sap_Metric_Learning! learning_rate=0.0001 weight_decay=0.01 use_cuda=True loss=ms_loss use_miner=True miner_margin=0.2 type_of_triplets=all agg_mode=cls ]


miner: TripletMarginMiner(
  (distance): LpDistance()
)
loss: MultiSimilarityLoss(
  (distance): CosineSimilarity()
  (reducer): MeanReducer()
)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:root:Epoch 1/1
10/31/2023 11:54:43 AM: [ Epoch 1/1 ]
INFO:root:train!
10/31/2023 11:54:43 AM: [ train! ]
100%|██████████| 1415/1415 [09:04<00:00,  2.60it/s]
INFO:root:loss/train_per_epoch=0.5613486295028871/1
10/31/2023 12:03:48 PM: [ loss/train_per_epoch=0.5613486295028871/1 ]
INFO:root:Training Time!0 hours 9 minutes 8 seconds
10/31/2023 12:03:51 PM: [ Training Time!0 hours 9 minutes 8 seconds ]
